# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

#### 1. Two sample hypothesis test is appropriate for this problem.
####     CLT is applicable here because there are more than 30 observations in the data.

#### 2. Null Hypothesis: The callback percentages of Whites and Blacks are equal.

####     Alternate Hypothesis: The callback percentage of Whites and Blacks is different.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [7]:
df = data[['race','call']]
df.head()

,race,call
0,w,0.0
1,w,0.0
2,b,0.0
3,b,0.0
4,w,0.0


####  Percentage of Black people getting call backs

In [8]:
bc = sum(data[data.race=='b'].call)
b = len(data[data.race=='b'])
bpercent = bc/b
bpercent*100

6.447638603696099

####  Percentage of White people getting call backs

In [9]:
wc = sum(data[data.race=='w'].call)
w = len(data[data.race=='w'])
wpercent = wc/w
wpercent*100

9.650924024640657

In [10]:
# Sepating blacks data

bdf = df[df.race=='b']
bdf.head()

,race,call
2,b,0.0
3,b,0.0
7,b,0.0
8,b,0.0
9,b,0.0


In [11]:
# Sepating whites data

wdf = df[df.race=='w']
wdf.head()

,race,call
0,w,0.0
1,w,0.0
4,w,0.0
5,w,0.0
6,w,0.0


### 3. Bootstrap Hypothesis Test - confidence interval

In [12]:
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))


def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [23]:
# def percentage(data):
#     callbacks = data['call'].sum()
#     length = len(data)
#     percent = callbacks/length
#     return percent * 100



#############    In this case, percentage should be the same as mean*100

In [29]:
# Draw bootstrap replicates of the percentage of callback: bs_replicates
bs_rep = draw_bs_reps(bdf['call'], np.mean, size=10000)

# bs_rep
bs_rep = [x * 100 for x in bs_rep]
# bs_rep

# Compute the 95% confidence interval: conf_int
conf_int = np.percentile(bs_rep, [2.5, 97.5])

# Print the confidence interval
print('95% confidence interval =', conf_int, 'games')

95% confidence interval = [5.50205335 7.4332647 ] games


### p-value from bootstrap replicates

In [30]:
# difference in callback percentage for white and black from given data
empirical_diff_percentage = wpercent*100 - bpercent*100

In [34]:
# replicating bootstrap samples
w_bs_rep = draw_bs_reps(wdf['call'], np.mean, size=10000)
b_bs_rep = draw_bs_reps(bdf['call'], np.mean, size=10000)


# multiplying both arrays with 100 to convert to percentages
w_bs_rep = [x * 100 for x in w_bs_rep]
b_bs_rep = [x * 100 for x in b_bs_rep]


# difference array - first converting to numpy array and then subtracting
diff = np.array(w_bs_rep) - np.array(b_bs_rep)


# Compute and print p-value: p
p = np.sum(diff >= empirical_diff_percentage) / len(diff)
print('p-value =', p)

p-value = 0.5089


### 4.

#### Since the p-value > 0.05, there is not a strong evidence against the null hypothesis. Therefore it cn be concluded that there is not a significant difference in the call back percentages for whites and blacks.

### 5. Conclusion

#### The above analysis suggests that race/name is NOT the most important factor in callback success.